In [1]:
pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.1 MB/s eta 0:00:00


In [7]:
import os
from groq import Groq

# Replace with your actual Groq API Key
os.environ["GROQ_API_KEY"] = "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"

# Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def classify_emotion(text):
    if not client or not text:
        print("⚠️ Missing client or text.")
        return "Unavailable"

    prompt = f"""
You are an emotion classification assistant.

Task: Read the following sentence and identify all relevant emotions being expressed.

Respond with a comma-separated list of emotions from this list:

happy, sad, anxious, angry, calm, stressed, excited, neutral, lonely, tired, confused, surprised, grateful, frustrated, hopeful, bored

Sentence: "{text}"
Emotions:
"""

    try:
        print("🤖 Sending to Groq...")
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama3-8b-8192"
        )
        raw_output = response.choices[0].message.content.strip().lower()

        # Split the response into a list of emotions and filter for valid ones
        valid_emotions = ["happy", "sad", "anxious", "angry", "calm", "stressed", "excited", "neutral", "lonely", "tired", "confused", "surprised", "grateful", "frustrated", "hopeful", "bored"]
        detected_emotions = [e.strip() for e in raw_output.split(',') if e.strip() in valid_emotions]

        if detected_emotions:
            print(f"🎯 Detected Emotions: {', '.join(detected_emotions)}")
            return detected_emotions
        else:
            print("🎯 No valid emotions detected.")
            return ["unknown"]

    except Exception as e:
        print("❌ Emotion classification failed:", e)
        return ["Error"]


# Example
if __name__ == "__main__":
    text = input("Enter user message: ")
    emotion = classify_emotion(text)
    print(f"Detected Emotion: {emotion}")


Enter user message: I had a very hard at work today
🤖 Sending to Groq...
🎯 Detected Emotions: stressed, frustrated
Detected Emotion: ['stressed', 'frustrated']


#Voice based sentiment analysis

# Task
Perform sentiment analysis on voice input by recording or loading an audio file, transcribing it to text, and then analyzing the text for sentiment using a language model.

In [8]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch groq
!apt-get install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ebbm75x4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ebbm75x4
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━

In [9]:
# 🧠 Step 2: Upload Audio File
from google.colab import files

uploaded = files.upload()
AUDIO_FILE = list(uploaded.keys())[0]  # Use uploaded file
print(f"📂 Uploaded file: {AUDIO_FILE}")


Saving analysis2.m4a to analysis2.m4a
📂 Uploaded file: analysis2.m4a


In [10]:
# 🧠 Step 3: Load Whisper Model
import whisper

try:
    whisper_model = whisper.load_model("base")  # Options: tiny, base, small, medium, large
    print("✅ Whisper model loaded.")
except Exception as e:
    print("❌ Failed to load Whisper:", e)


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.2MiB/s]


✅ Whisper model loaded.


In [11]:
# 📝 Step 4: Transcribe Audio to Text
def transcribe_audio(filename):
    print(f"🎙 Transcribing {filename}...")
    try:
        result = whisper_model.transcribe(filename)
        print(f"💬 Transcribed Text: {result['text']}")
        return result["text"]
    except Exception as e:
        print("❌ Transcription failed:", e)
        return None


In [12]:
# 🤖 Step 5: Initialize Groq Client
from groq import Groq

# Replace with your actual Groq API key
GROQ_API_KEY = "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"  # ← 🔐 Paste your key here
MODEL_NAME = "llama3-8b-8192"       # or "llama3-70b-8192"

try:
    client = Groq(api_key=GROQ_API_KEY)
    print("✅ Groq client initialized.")
except Exception as e:
    print("❌ Error initializing Groq:", e)
    client = None


✅ Groq client initialized.


In [17]:
def classify_emotion(text):
    if not client or not text:
        print("⚠️ Missing client or text.")
        return "Unavailable"

    prompt = f"""
You are an emotion classification assistant.
Given a user's message, classify the main emotion they are expressing.
Choose one from: happy, sad, anxious, angry, calm, stressed, excited, neutral, lonely, tired, confused, surprised, grateful, frustrated, hopeful, bored
Message: "{text}"
Emotion:"""

    try:
        print("🤖 Sending to Groq...")
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=MODEL_NAME
        )
        emotion = response.choices[0].message.content.strip().lower().split()[0].strip('.,')
        print(f"🎯 Detected Emotion: {emotion}")
        return emotion
    except Exception as e:
        print("❌ Emotion classification failed:", e)
        return "Error"


In [16]:
# 🚀 Step 7: Run Full Pipeline
text = transcribe_audio(AUDIO_FILE)
if text:
    classify_emotion(text)
else:
    print("⚠️ Transcription failed. Cannot proceed.")


🎙 Transcribing analysis2.m4a...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


💬 Transcribed Text:  Hi, I'm feeling very sad and anxious today and I don't want to do any work.
🤖 Sending to Groq...
🎯 Detected Emotion: based
